# Lab02 - Pendulum - Lab Bench

Load relevant packages with the code below and check to make sure you are working in the right directory.

In [ ]:
%pip install ipympl ipywidgets

In [ ]:
%matplotlib widget
# Imports used throughout the lab (arrays, plotting, data frames, ODE integration)
from pathlib import Path

import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.integrate import solve_ivp
from datetime import datetime

notebook_loc = Path(os.getcwd())
# check that has the right parent
if notebook_loc.parent.name != 'work':
  print('Please make a copy of this notebook and put it in your "work" folder!')
else:
  fig_dir = notebook_loc/'figures'
  fig_dir.mkdir(parents=True, exist_ok=True)
  print('Great, your figures will be saved in:', fig_dir)

def fig_save(fig_name, filetype='png'):
  fig_path = fig_dir/f'{fig_name}.{filetype}'
  unique_fig_path = fig_dir/f'{fig_path.name}__{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}{fig_path.suffix}'
  plt.savefig(fig_path, dpi=300)
  plt.savefig(unique_fig_path, dpi=300)

sys.path.append(str(notebook_loc/'support_code'))
from phase_portraits import *

In [ ]:
def pendulum(t, u, damping):
    theta, omega = u
    dtheta_dt = omega
    domega_dt = -damping * omega - np.sin(theta)
    return [dtheta_dt, domega_dt]

def convert2circle(u):
    u = u.copy()
    u[0,:] = np.mod(u[0,:]+np.pi, 2*np.pi) - np.pi
    jumps = np.abs(np.diff(u[0,:]))
    ind = 1 + np.where(jumps > np.pi)[0]
    return np.insert(u, ind, np.nan, axis=1)

## Investigation

### A. Simple Pendulum

#### Time Domain

##### Load Data

Import your tracked pendulum data as a table or build the table yourself. The columns should be `time` (in seconds) and `theta` (release angle in degrees). Make sure angles are in degrees.

In [ ]:
df_period = pd.read_csv(notebook_loc/'data'/"pen_data_lab.csv",index_col=0)
df_period['theta_rad'] = df_period['theta'].apply(np.deg2rad)
df_period = df_period.rename(columns={'T':'period'})
df_period['source'] = 'observation'

##### Compute Period

Compute $T_\text{lin}$ from your measured length $L$

In [ ]:
L = 0.42  # meters (replace)
g = 
T_lin = 2*np.pi*np.sqrt(L/g)
T_lin

##### Plot

Use the code below to plot your calculated value with your observed values.

In [ ]:
fig, ax = plt.subplots()

sns.scatterplot( data = df_period, x = 'theta',y='period', hue='source', ax=ax)
ax.axhline(y = T_lin, c='k', label='calculated')
ax.annotate('calculated', xy=(T_lin, 17), xytext=(1, 19),
            arrowprops=dict(facecolor='black', shrink=0.05),
            )
ax.set_ylabel('Period (s)')
ax.set_xlabel('Release angle, '+r'$\theta$'+ '(degrees)')
plt.tight_layout()
# fig_save('scatter_period_release_angle')

##### Simulate

Now use the code below to calculate the theoretical period of the undamped pendulum motion for these same release angles.

In [ ]:
from scipy.special import ellipk

omg0 = np.sqrt(g/L) # fundamental angular frequency
Ep = 4*omg0**2 #maxmimum potential energy

def calc_simulated_period(theta)
    if theta > np.pi/2:
        theta = np.deg2rad(theta)

    E0 = 0 + Ep*np.sin(theta/2)**2  
    k = np.sqrt(E0/Ep)
    return 4*ellipk(k**2)/omg0  # beware: the elliptic integral's parameter is the square of the energy ratio in this formulation

df_period_sim = pd.DataFrame({'theta': df_period['theta'], 'period': df_period['theta'].apply(calc_simulated_period)})
df_period_sim['source'] = 'simulated'

df_period = pd.concat([df_period, df_period_sim])

To produce your deliverable plot, re-run the plotting cell with this updated dataframe.

#### Phase Space

In class, we touched on how it can be easier to characterize or infer the structure of a system by looking at it in phase space versus in the time domain.

Now let’s construct a phase portrait.

First, specify some angles not covered in your obsevations. The code expects the values to be in degrees.

In [ ]:
sim_angles = [] # in degrees, the code will handle radians as necessary

Next, simulate trajectories for all initial angles (both measured and supplemental)

In [ ]:
Neval = 10
Nframes=80
t_end = 0.1*Nframes
t_eval=np.linspace(0, t_end, Neval*Nframes)
damping=0

sol_d = {}
for ip, angle_m in enumerate(df_period['theta'].values.tolist()+sim_angles):
    theta_dot_init=0
    sol = solve_ivp(pendulum, [0, t_end], [np.deg2rad(angle_m), theta_dot_init], t_eval=t_eval,
                                        args=[damping], atol=1.e-8, rtol=1.e-6)
    sol_d[angle_m]=sol

The code below will generate a (1) a phase portrait and (2) a reproduction of the release angle-period plot you made previously.

In [ ]:
fig, axs = plt.subplot_mosaic([['phase_portrait'], ['scatter']],
                              figsize=(6, 9),
                              height_ratios=(3, 1),
                              layout='constrained')

sc = axs['scatter'].scatter(x='theta', y='period', data=df_period, c='theta_rad')
axs['scatter'].set(ylabel = 'Period (s)', xlabel=r'Release angle, $\theta_0$')

Neval = 10
Nframes=80
t_end = 0.1*Nframes
t_eval=np.linspace(0, t_end, Neval*Nframes)
damping=0

cmap = sc.get_cmap()
norm = sc.get_array()
find_color = lambda x: cmap((x - norm.min()) / (norm.max() - norm.min()))

sol_d = {}
for ip, angle_m in enumerate(df_period['theta'].values.tolist()+sim_angles):
    theta_dot_init=0
    sol = solve_ivp(pendulum, [0, t_end], [np.deg2rad(angle_m), theta_dot_init], t_eval=t_eval,
                                        args=[damping], atol=1.e-8, rtol=1.e-6)
    sol_d[angle_m]=sol

sim_period = []
for angle_m in sol_d.keys():
    sol =  sol_d[angle_m]
    soln = convert2circle(sol.y)

    k = np.where(soln[1] > 0)[0][0]*2 - 1
    axs['phase_portrait'].plot(sol.y[0][:k],
             sol.y[1][:k], alpha=.5, color=find_color(np.deg2rad(angle_m)))

    if angle_m in sim_angles:
      sim_period.append(calc_simulated_period(angle_m))
      theta0 = np.deg2rad(angle_m)
      
      tt = t_eval[:k]
      theta_comp = theta0 * np.cos(tt)
      omega_comp = -theta0  * np.sin(tt)

      axs['phase_portrait'].plot(theta_comp, omega_comp, '--', alpha=.2,  color='k')

axs['phase_portrait'].set(ylabel=r'd$\theta$/dt', xlabel= r'$\theta$')
axs['phase_portrait'].set_aspect('equal')

axs['scatter'].scatter(sim_angles, np.array(sim_period), color='k', alpha=.4)
axs['scatter'].set_aspect('auto')

### B. Pendulum Systems

#### Constant driving force

Consider this generalized phase portrait.

Running the next cell will create the phase portrait of this model and an illustration of a physical pendulum. Clicking with the mouse in the phase portrait will simulate the pendulum motion starting with the initial conditions (angle, angular velocity) selected by the mouse click.

In [ ]:
def pendulum(t, u, damping):
    return [u[1], -damping*u[1] - np.sin(u[0])]


damping = 0
pp.generate_phase_portrait(pendulum, damping, Nframes=600)

#### Overcoming damping

#### 1D Chaos